In [ ]:
#Imports
import pandas as pd
import networkx as nx
import time
import tweepy
import datetime

In [ ]:
#API Keys
consumer_key = 'HoshUlv1vavF5iYgdNk6TmaOM'
consumer_secret = 'tvFIN4wsy3CnjYWpPX88vLMxb7i5BfLrMyJSm0trPly3R6iBDO'
access_token = '796224491395366914-Tn2B2iBpy6e5xDj5GquZw5P8ap7Lpdi'
access_token_secret = '3csIFTivfgeSP8iEvtOhEzPUH1Pxqzvl6h5gCX0OA4IKQ'

In [ ]:
# load data
digwell = pd.read_pickle("digwell.pkl")
digwell.set_index("id", inplace=True)

In [ ]:
# timeframe
today = datetime.date.today()

start_date = today - datetime.timedelta(days = today.weekday(), weeks = 1)
end_date   = start_date + datetime.timedelta(days = 7)

def utc2snowflake(utc_timestamp):
    return (int(round(utc_timestamp * 1000)) - 1288834974657) << 22

max_id = utc2snowflake(time.mktime(end_date.timetuple()))
min_id = utc2snowflake(time.mktime(start_date.timetuple()))

In [ ]:
# graph
G = nx.DiGraph() 
G.add_nodes_from(list(digwell.index))
cols = list(digwell.columns)
cols.remove("object")
for col in cols:
   nx.set_node_attributes(G, pd.Series(digwell[col]).to_dict(), col)

In [ ]:
# iterating through timeline for mentions

ids = list(digwell.index)
ids = list(map(int, ids))

failed = []
j = 0

for i in ids:
    j += 1
    print(str(i) + " " + str(j))
    try:
        search = api.user_timeline(user_id = i, min_id = min_id, max_id = max_id, include_rts = True)
        for tweet in search:
            user_mentions = tweet.entities['user_mentions']
            for user in user_mentions:
                mention = list(user['id_str'])
                digwell_mention = list(set(ids) & set(mention))
                joined = list(zip([i] * len(digwell_mention), digwell_mention))
                G.add_edges_from(joined)
                nx.write_gpickle(G, "digwelltwitterment.gpickle")
    except tweepy.error.TweepError as ex:
        print(ex.reason)
        failed.append(i)

In [ ]:
G.add_edges_from(joined)
nx.write_gpickle(G, "digwelltwitterment.gpickle")

In [ ]:
# save edgelist 
edge_df = nx.to_pandas_edgelist(G)
edge_df.to_csv("digwell_mentions_edges.csv", index = False) 